# Dijkstras 

It runs on directed and weighted graph. It is similar to prims and BFS. It is similar to BFS because it uses min priority queue. Similar to Prims because it uses the growing tree concept.



In [32]:
import math
import sys

class Vertex:
    def __init__(self, node, weight=0):
        self.id = node
        self.visited = False
        self.adjList = {}
        self.weight = weight
        self.distance = sys.maxsize
        self.color = 'WHITE'
        self.parent = None
        self.start_time = -1
        self.finish_time = -1

    def set_start_time(self, start_time):
        self.start_time = start_time

    def get_start_time(self):
        return self.start_time

    def get_finish_time(self):
        return self.finish_time

    def set_finish_time(self, finish_time):
        self.finish_time = finish_time

    def add_neighbour(self, node, weight=0):
        self.adjList[node] = weight

    def set_distance(self, distance):
        self.distance = distance

    def get_distance(self):
        return self.distance

    def set_visited(self):
        self.visited = True

    def get_visited(self):
        return self.visited

    def set_id(self, node):
        self.id = node

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjList[neighbor]

    def get_neighbours(self):
        return self.adjList

    def set_parent(self, parent):
        self.parent = parent

    def get_parent(self):
        return self.parent

    def set_color(self, color):
        self.color = color

    def get_color(self):
        return self.color


class GraphAL():
    def __init__(self, isDag=False):
        self.numVertices = 0
        self.isDag = isDag
        self.vertices = {}

    def add_vertex(self, ID):
        self.numVertices += 1
        new_vertex = Vertex(ID)
        self.vertices[ID] = new_vertex

    def add_edge(self, u, v, weight):
        if u not in self.vertices:
            self.add_vertex(u)

        if v not in self.vertices:
            self.add_vertex(v)

        self.vertices[u].add_neighbour(self.vertices[v], weight)

        if not self.isDag:
            self.vertices[v].add_neighbour(self.vertices[u], weight)

    def get_vertex(self, node):
        if node in self.vertices:
            return self.vertices[node]

        return -1

    def get_vertices(self):
        return self.vertices

    def get_edges(self):
        edges = []
        for u in self.vertices.values():
            for v in u.get_neighbours():
                edges.append((u.get_id(), v.get_id(), u.get_weight(v)))

        return (edges)


G = GraphAL()
G.add_vertex('a')
G.add_vertex('b')
G.add_vertex('c')
G.add_vertex('d')
G.add_vertex('e')

G.add_edge('a', 'e', 10)
G.add_edge('a', 'c', 20)
G.add_edge('c', 'b', 30)
G.add_edge('b', 'e', 40)
G.add_edge('e', 'd', 50)
G.add_edge('f', 'e', 60)

print(G.get_vertices())
G.get_edges()

{'a': <__main__.Vertex object at 0x10794b748>, 'b': <__main__.Vertex object at 0x10e50f4a8>, 'c': <__main__.Vertex object at 0x10e50f2e8>, 'd': <__main__.Vertex object at 0x10e50f0f0>, 'e': <__main__.Vertex object at 0x10e50f3c8>, 'f': <__main__.Vertex object at 0x10e50f1d0>}


[('a', 'e', 10),
 ('a', 'c', 20),
 ('b', 'c', 30),
 ('b', 'e', 40),
 ('c', 'a', 20),
 ('c', 'b', 30),
 ('d', 'e', 50),
 ('e', 'a', 10),
 ('e', 'b', 40),
 ('e', 'd', 50),
 ('e', 'f', 60),
 ('f', 'e', 60)]

In [35]:
def relax(u, v):
    
    v.set_distance()


import heapq, sys


def dijkstra(G, s):
    arr = [(s, 0)]
    for u in G.get_vertices().values():
        if u.get_id() not in arr[0]:
            arr.append((u.get_id(), sys.maxsize))

    print(arr)
    heapq.heapify(arr)

    while arr:
        u = heapq.heappop(arr)
        print(u)
        for v in G.get_vertex(u[0]).get_neighbours():
            relax(G.get_vertex(u[0]), v)


dijkstra(G, 'a')

[('a', 0), ('b', 9223372036854775807), ('c', 9223372036854775807), ('d', 9223372036854775807), ('e', 9223372036854775807), ('f', 9223372036854775807)]
('a', 0)
('b', 9223372036854775807)
('c', 9223372036854775807)
('d', 9223372036854775807)
('e', 9223372036854775807)
('f', 9223372036854775807)


In [21]:
print(heapq.heapify([1, 0, 1]))

None


In [42]:
a = list(G.get_vertices().values())
print(a)
heapq.heapify(a)
print(a)

[<__main__.Vertex object at 0x10794b748>, <__main__.Vertex object at 0x10e50f4a8>, <__main__.Vertex object at 0x10e50f2e8>, <__main__.Vertex object at 0x10e50f0f0>, <__main__.Vertex object at 0x10e50f3c8>, <__main__.Vertex object at 0x10e50f1d0>]


TypeError: '<' not supported between instances of 'Vertex' and 'Vertex'